In [1]:
import pandas as pd
import pickle
import json

In [2]:
with open('dialogos.pickle', "rb") as f:
    dialogos_completos = pickle.load(f)

In [ ]:
#szigriszt_pazos cuanto más cerca de 100, más sencillo es

In [4]:
df = pd.DataFrame(dialogos_completos)
df['hablante'] = df['hablante'].apply(json.dumps)
df = pd.concat([df, pd.json_normalize(df['hablante'].apply(json.loads))], axis=1).drop(columns=['hablante'])
df

,discurso,fecha_proceso,szigriszt_pazos,flesch_reading_ease,nombre,grupo parlamentario
0,"Señor Sánchez, usted bloquea hasta la verdad ...",2020-02-19,-79.60,-79.01,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso
1,"Muchas gracias, señora presidenta Claro que n...",2020-02-19,32.24,35.84,"Sánchez Pérez-Castejón, Pedro",Grupo Parlamentario Socialista
2,"Señor Sánchez, sus recetas económicas son tan...",2020-02-19,-60.71,-59.39,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso
3,"Muchas gracias, señora presidenta Señor Casad...",2020-02-19,-78.36,-77.63,"Sánchez Pérez-Castejón, Pedro",Grupo Parlamentario Socialista
4,"Gracias, señora presidenta Señor Sánchez, el ...",2020-02-19,-0.88,2.24,"Arrimadas García, Inés",Grupo Parlamentario Ciudadanos
...,...,...,...,...,...,...
19157,"Termino Como le dije, doña Isabel, su discurs...",2021-05-11,74.03,78.20,"Mestre Barea, Manuel",Grupo Parlamentario VOX
19158,"Muchas gracias, señora presidenta Señorías, m...",2021-05-11,-804.20,-817.55,"Gutiérrez Díaz de Otazu, Fernando Adolfo",Grupo Parlamentario Popular en el Congreso
19159,con las mayores garantías de éxito y de segur...,2021-05-11,50.67,55.52,"Gutiérrez Díaz de Otazu, Fernando Adolfo",Grupo Parlamentario Popular en el Congreso
19160,"Gracias, señora presidenta Señorías, antes de...",2021-05-11,-65.56,-64.45,"Cerdán León, Santos",Grupo Parlamentario Socialista


In [45]:
resultados = df.groupby(['grupo parlamentario']).agg({'szigriszt_pazos': ['max', 'min']}).reset_index()

# renombramos las columnas del resultado
resultados.columns = ['grupo parlamentario', 'max_szigriszt_pazos', 'min_szigriszt_pazos']

# hacemos un merge de los resultados con el dataframe original para conservar todas las columnas
resultados = pd.merge(df, resultados, on='grupo parlamentario')


In [46]:
medias = df.groupby(['grupo parlamentario', 'fecha_proceso'])['szigriszt_pazos'].mean().reset_index()


In [39]:
lista_grupos = ['Grupo Parlamentario Ciudadanos', 
                'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común',
                'Grupo Parlamentario Popular en el Congreso',
                'Grupo Parlamentario Socialista',
                'Grupo Parlamentario VOX'
               ]
df_medias = medias[medias['grupo parlamentario'].isin(lista_grupos)]

In [107]:
import os
import re
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook

output_notebook()  # para mostrar la gráfica en el notebook

    
colores_grupos = {
    'GCs':'#DF6524',
    'GCUP_EC_GC':'#693279',
    'GP': '#184A90',
    'GS': '#DC0612',
    'GVOX': '#63BE21'
}
    

# Crear un objeto ColumnDataSource con los datos
source_GCs = ColumnDataSource(data=dict(
    fecha=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario Ciudadanos']['fecha_proceso'],
    szigriszt_pazos=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario Ciudadanos']['szigriszt_pazos']
))

source_GCUP_EC_GC = ColumnDataSource(data=dict(
    fecha=df_medias[df_medias['grupo parlamentario'] == 
                    'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común']['fecha_proceso'],
    szigriszt_pazos=df_medias[df_medias['grupo parlamentario'] == 
                        'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común']['szigriszt_pazos']
))

source_GP = ColumnDataSource(data=dict(
    fecha=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario Popular en el Congreso']['fecha_proceso'],
    szigriszt_pazos=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario Popular en el Congreso']['szigriszt_pazos']
))

source_GS = ColumnDataSource(data=dict(
    fecha=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario Socialista']['fecha_proceso'],
    szigriszt_pazos=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario Socialista']['szigriszt_pazos']
))

source_GVOX = ColumnDataSource(data=dict(
    fecha=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario VOX']['fecha_proceso'],
    szigriszt_pazos=df_medias[df_medias['grupo parlamentario'] == 'Grupo Parlamentario VOX']['szigriszt_pazos']
))

# Crear una figura de Bokeh
p = figure(title="Complejidad de discurso por fecha", x_axis_label="Fecha", y_axis_label="szigriszt_pazos",
           x_range=df_medias["fecha_proceso"].drop_duplicates().sort_values(ascending=True), plot_width=2400, plot_height=600)
p.xaxis.major_label_orientation = 45

# Añadir la línea a la figura
p.line(x="fecha", y="szigriszt_pazos", source=source_GCs, line_color=colores_grupos['GCs'], line_width=2, legend_label="GCs")
p.line(x="fecha", y="szigriszt_pazos", source=source_GCUP_EC_GC, line_color=colores_grupos['GCUP_EC_GC'], line_width=2, legend_label="GCUP_EC_GC")
p.line(x="fecha", y="szigriszt_pazos", source=source_GP, line_color=colores_grupos['GP'], line_width=2, legend_label="GP")
p.line(x="fecha", y="szigriszt_pazos", source=source_GS, line_color=colores_grupos['GS'], line_width=2, legend_label="GS")
p.line(x="fecha", y="szigriszt_pazos", source=source_GVOX, line_color=colores_grupos['GVOX'], line_width=2, legend_label="GVOX")

# Añadir la leyenda a la figura
p.legend.location = "bottom_right"
p.legend.click_policy="hide"

# Mostrar la figura
show(p)

Loading BokehJS ...

In [47]:
# 20.02.2020 DELEGACIÓN EN LA MESA DE LA COMISIÓN DE LA COMPETENCIA DE ESTA DE ADOPCIÓN DE LOS ACUERDOS A QUE SE REFIERE EL ARTÍCULO 44 DEL REGLAMENTO, CONCORDANTE CON LA RESOLUCIÓN DE LA PRESIDENCIA DE LA CÁMARA DE 2 DE NOVIEMBRE DE 1983.
# 18.03.2020 Comparecencia del presidente del Gobierno en el Congreso de los Diputados por el Estado de Alarma en la crisis del coronavirus. .
# 25.03.2020 E SE DEROGA EL DESPIDO OBJETIVO POR FALTAS DE ASISTENCIA AL TRABAJO ESTABLECIDO EN EL ARTÍCULO 52. D) DEL TEXTO REFUNDIDO DE LA LEY DEL ESTATUTO DE LOS TRABAJADORES,
# 09.04.2020 Prórroga del estado de alarma
# 16.05.2020 Prórroga del estado de alarma
# 14.07.2020 Proyecto de ley de cambio climático y transición energética.
# 20.10.2020 Moción de censura, candidato SAntiago Abascal
# 28.01.2021 Proposición de Ley Orgánica por la que se modifica la Ley Orgánica 6/1985, de 1 de julio, del Poder Judicial, para el establecimiento del régimen jurídico aplicable al Consejo General del Poder Judicial en funciones.
#

In [15]:
import textstat
textstat.set_lang('es')
textstat.szigriszt_pazos('hola, ¿como te llamas?')

93.82

In [17]:
texto = 'La polimerización se detiene destruyendo los radicales libres presentes por medio de la adición de un estabilizante de acción rápida. Luego de remover el monómero que no ha reaccionado, el polímero es asilado desestabilizando el sistema coloidal, separando la fase acuosa y secando el polímero.'
textstat.szigriszt_pazos(texto)

38.97

Un valor negativo en la media del valor de Szigriszt-Pazos de un grupo parlamentario indica que los discursos de los diputados de ese grupo parlamentario son en promedio difíciles de leer y comprenden para una audiencia general. Esto podría deberse a varios factores, como el uso de un lenguaje técnico o jerga, la complejidad de las oraciones o la falta de claridad en la exposición.

En un informe, podrías destacar que el valor negativo en la media de Szigriszt-Pazos sugiere que los discursos de los diputados de ciertos grupos parlamentarios pueden no estar comunicando efectivamente sus ideas a una audiencia amplia, lo que podría tener un impacto negativo en su capacidad para persuadir a otros y para obtener el apoyo público.

También podrías mencionar que la media del valor de Szigriszt-Pazos por sí sola no proporciona información completa sobre la calidad de los discursos de los diputados, y que es importante considerar otros factores, como el contenido y la relevancia de las ideas expresadas en los discursos, para hacer una evaluación completa de su efectividad.


In [85]:
import os
import re
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook

output_notebook()  # para mostrar la gráfica en el notebook

folder_path = 'resultados_analisis'  # Reemplaza por la ruta a la carpeta que deseas leer
#pattern = r'^COM_DISCURSO_(\d{4})_(\d{2})_(\d{2})_GS\.csv$'  # Patrón para filtrar los archivos y capturar los grupos de fecha
pattern = r'^COM_DISCURSO_(\d{4})_(\d{2})_(\d{2})_(GCs|GCUP_EC_GC|GP|GS|GVOX)\.csv$'

# Obtiene los nombres de todos los archivos en la carpeta
files = os.listdir(folder_path)

# Filtra los archivos que cumplan con el patrón
filtered_files = [f for f in files if re.match(pattern, f)]

df_salida = pd.DataFrame()
# Itera sobre los archivos filtrados
for file_name in filtered_files:
    try:
        file_path = os.path.join(folder_path, file_name)
        polaridades = pd.read_csv(file_path)
        match = re.match(pattern, file_name)
        year = match.group(1)
        month = match.group(2)
        day = match.group(3)
        grupo = match.group(4)
        date_str = f"{year}-{month}-{day}"
        polaridades['fecha'] = date_str
        polaridades['grupo'] = grupo
        media_polaridad = polaridades.groupby(['grupo', 'fecha'])['polaridad'].mean()
        media_polaridad_df = media_polaridad.reset_index()
        df_salida = pd.concat([df_salida, media_polaridad_df], axis=0)
    except pd.errors.EmptyDataError:
        continue
    
colores_grupos = {
    'GCs':'#DF6524',
    'GCUP_EC_GC':'#693279',
    'GP': '#184A90',
    'GS': '#DC0612',
    'GVOX': '#63BE21'
}
    

# Crear un objeto ColumnDataSource con los datos
source_GCs = ColumnDataSource(data=dict(
    fecha=df_salida[df_salida['grupo'] == 'GCs']['fecha'],
    polaridad=df_salida[df_salida['grupo'] == 'GCs']['polaridad']
))

source_GCUP_EC_GC = ColumnDataSource(data=dict(
    fecha=df_salida[df_salida['grupo'] == 'GCUP_EC_GC']['fecha'],
    polaridad=df_salida[df_salida['grupo'] == 'GCUP_EC_GC']['polaridad']
))

source_GP = ColumnDataSource(data=dict(
    fecha=df_salida[df_salida['grupo'] == 'GP']['fecha'],
    polaridad=df_salida[df_salida['grupo'] == 'GP']['polaridad']
))

source_GS = ColumnDataSource(data=dict(
    fecha=df_salida[df_salida['grupo'] == 'GS']['fecha'],
    polaridad=df_salida[df_salida['grupo'] == 'GS']['polaridad']
))

source_GVOX = ColumnDataSource(data=dict(
    fecha=df_salida[df_salida['grupo'] == 'GVOX']['fecha'],
    polaridad=df_salida[df_salida['grupo'] == 'GVOX']['polaridad']
))

# Crear una figura de Bokeh
p = figure(title="Polaridad por fecha", x_axis_label="Fecha", y_axis_label="Polaridad",
           x_range=df_salida["fecha"].drop_duplicates().sort_values(ascending=True), plot_width=2400, plot_height=600)
p.xaxis.major_label_orientation = 45

# Añadir la línea a la figura
p.line(x="fecha", y="polaridad", source=source_GCs, line_color=colores_grupos['GCs'], line_width=3, legend_label="GCs")
p.line(x="fecha", y="polaridad", source=source_GCUP_EC_GC, line_color=colores_grupos['GCUP_EC_GC'], line_width=3, legend_label="GCUP_EC_GC")
p.line(x="fecha", y="polaridad", source=source_GP, line_color=colores_grupos['GP'], line_width=3, legend_label="GP")
p.line(x="fecha", y="polaridad", source=source_GS, line_color=colores_grupos['GS'], line_width=3, legend_label="GS")
p.line(x="fecha", y="polaridad", source=source_GVOX, line_color=colores_grupos['GVOX'], line_width=3, legend_label="GVOX")

# Añadir la leyenda a la figura
p.legend.location = "bottom_right"
p.legend.click_policy="hide"

# Mostrar la figura
show(p)



Loading BokehJS ...